In [ ]:
!pip install missingno xgboost imbalanced-learn

In [ ]:
#@title lib install
import pandas as pd
import numpy as np 
import missingno as msno
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix



In [ ]:
from google.colab import drive
drive.mount("/content/drive")
df = pd.read_csv(r"/content/drive/MyDrive/BIDV/CARD_FRAUD/data/all_data_with_header.csv")
df.shape

In [ ]:
df.head()

In [ ]:
target = df['MERCH_FR']

In [ ]:
print('kiểu dữ liệu trong tập')
print(df.dtypes.unique())
print(df.dtypes.nunique())
print (df.isnull().values.any())
print (df.isna().sum())

cols_missing_val = df.columns[df.isnull().any()].tolist()
print(cols_missing_val)

In [ ]:
msno.bar(df[cols_missing_val_train],figsize=(20,8),color="#19455e",fontsize=18,labels=True)
#msno.matrix(df_train[cols_missing_val_train],width_ratios=(10,1),figsize=(20,8),color=(0.2,0.2,0.2),fontsize=18,sparkline=True,labels=True)

In [ ]:
#------- check memory usage---------
mem = df.memory_usage(index=True).sum()
print("Memory consumed by dataset  :   {} MB" .format(mem/ 1024**2))
def change_datatype_int(df):
    int_cols = list(df.select_dtypes(include=['int']).columns)
    for col in float_cols:
        if ((np.max(df[col]) <= 127) and(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)

def change_datatype_float(df):
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)

In [ ]:
change_datatype_int(df)
change_datatype_float(df)

In [ ]:
# Class proportions are the SAME in y_train and y_test. (This is good!)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0, stratify=y)

In [ ]:
#ColumnTransfromer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
ct = ColumnTransformer(
    [('encoder', ohe, ['Embarked', 'Sex']),
     ('imputer', imp, ['Age'])],
    remainder='passthrough')
pipe = Pipeline([('preprocessor', ct), ('classifier', clf)])

# ct = make_column_transformer((ohe, make_column_selector(dtype_include=object)))
# column transformer with make_column_selector?

# set up preprocessing for numeric columns
imp_median = SimpleImputer(strategy='median', add_indicator=True)
scaler = StandardScaler()

# set up preprocessing for categorical columns
imp_constant = SimpleImputer(strategy='constant')
ohe = OneHotEncoder(handle_unknown='ignore')
# select columns by data type
num_cols = make_column_selector(dtype_include='number')
cat_cols = make_column_selector(dtype_exclude='number')

# do all preprocessing
preprocessor = make_column_transformer(
    (make_pipeline(imp_median, scaler), num_cols),
    (make_pipeline(imp_constant, ohe), cat_cols))

# create a pipeline
pipe = make_pipeline(preprocessor, LogisticRegression())